In [1]:
import sys
import wandb
import torch
sys.path.append('../../')
import pytorch_lightning as pl
from utils.data_modules.contrastive import EEGContrastiveDataModule
from models.trainers.contrastive import ContrastiveTrainerModel, EmbeddingSaveCallback

/workspace/eeg-image-decoding/data/all-joined-1/eeg/epochs


In [2]:
dm = EEGContrastiveDataModule(
    input_channels=['Fp1', 'AF7', 'AF3', 'F1', 'F3', 'F5', 'F7', 'FT7', 'FC5', 'FC3', 'FC1', 'C1', 'C3', 'C5', 'T7', 'TP7', 'CP5', 'CP3', 'CP1', 'P1', 'P3', 'P5', 'P7', 'P9', 'PO7', 'PO3', 'O1', 'Iz', 'Oz', 'POz', 'Pz', 'CPz', 'Fpz', 'Fp2', 'AF8', 'AF4', 'AFz', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FT8', 'FC6', 'FC4', 'FC2', 'FCz', 'Cz', 'C2', 'C4', 'C6', 'T8', 'TP8', 'CP6', 'CP4', 'CP2', 'P2', 'P4', 'P6', 'P8', 'P10', 'PO8', 'PO4', 'O2'],
    sfreq=250,
    montage='standard_1020',
    window_before_event_ms=50,
    window_after_event_ms=600,
    subject=1, 
    session=1, 
    batch_size=64, 
    num_workers=4,
    test='All'
)

In [3]:
sample_data = dm.get_sample_info()
epochs = 200
subject = 1
session = 1
num_channels = sample_data['input']['num_channels']
timesteps = sample_data['input']['num_timesteps']
num_fine_labels = sample_data['output']['fine_labels_shape']

Using ALL data for subject=1, session=1 as TEST SET
Found 3839 total samples for test set
Data split - Train: 2879, Val: 320, Test: 3839
Creating Datasets...
Original dataframe size: 2879
Original dataframe size: 320
Original dataframe size: 3839


In [4]:
checkpoint_path = "/workspace/eeg-image-decoding/code/models/check_points/contrastive_encoder/subj1_session1_epoch=199.ckpt"

In [6]:
lightning_model = ContrastiveTrainerModel.load_from_checkpoint(checkpoint_path)

results_callback = EmbeddingSaveCallback(save_dir="/workspace/eeg-image-decoding/data/all-joined-1/eeg/embeddings/650ms-250Hz/", filename=f"subj0{subject}_session{session}_eeg_embeddings.npy")

trainer = pl.Trainer(
    max_epochs=epochs,
    callbacks=[results_callback],
    accelerator='gpu' if torch.cuda.is_available() else 'cpu',
    devices=1
)

trainer.test(lightning_model, dm)

/venv/main/lib/python3.10/site-packages/braindecode/models/base.py:23: UserWarning: EEGNetv4: 'in_chans' is depreciated. Use 'n_chans' instead.
  warnings.warn(
/venv/main/lib/python3.10/site-packages/braindecode/models/base.py:23: UserWarning: EEGNetv4: 'n_classes' is depreciated. Use 'n_outputs' instead.
  warnings.warn(
/venv/main/lib/python3.10/site-packages/braindecode/models/base.py:23: UserWarning: EEGNetv4: 'input_window_samples' is depreciated. Use 'n_times' instead.
  warnings.warn(
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/venv/main/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a depe

Datasets already initialized, skipping setup


Testing: |          | 0/? [00:00<?, ?it/s]


=== TEST RESULTS ===
Test Precision: 0.0339
Test Recall: 0.7720
Test F1-Score: 0.0601
Test AUROC: 0.5016

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       test_auroc           0.5016080141067505
         test_f1            0.06014416366815567
     test_precision         0.03387201577425003
       test_recall          0.7720038890838623
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_precision': 0.03387201577425003,
  'test_recall': 0.7720038890838623,
  'test_f1': 0.06014416366815567,
  'test_auroc': 0.5016080141067505}]